In [ ]:
%load_ext autoreload
%autoreload 2

from IPython.display import display, HTML, Javascript
display(HTML('''<style>.jp-Cell.jp-CodeCell .jp-OutputArea:has(.jp-OutputArea-output) {background-color: #f9f9f9;border: 1px solid #ddd;padding: 10px;border-radius: 5px;margin: 5px 0;}</style><style>.prompt, .jp-InputPrompt {display:none !important;}</style>'''))

# Importing & running a `Case`
In this notebook, we will spin up a ROMS-MARBL run using C-Star. In particular, we will:
- Create a [C-Star Case](https://c-star.readthedocs.io/en/latest/terminology.html#term-Case) from a pre-prepared [blueprint](https://c-star.readthedocs.io/en/latest/terminology.html#term-blueprint)(`Case.from_blueprint()`). See [this notebook]() for instructions on how to assemble a blueprint.
- Examine the contents of the Case object we just created
- Set up the case locally (`Case.setup()`)
- Compile any necessary code associated with the case (`Case.build()`)
- Complete any pre-processing steps associated with the case (`Case.pre_run()`)
- Run the case with a small time step for a couple of days (`Case.run()`)
- Execute any post-processing steps associated with the case (`Case.post_run()`)

## Importing the `Case` 
[The "Case"](https://c-star.readthedocs.io/en/latest/generated/cstar.Case.html) is the primary object of C-Star, and contains all the information needed to run a particular simulation. Once prepared, cases can be stored in "blueprints" - `.yaml` files telling C-Star what goes into each case and where to find it. We will start from a blueprint that has been prepared in advance.

We can construct a `Case` from a blueprint using the `Case.from_blueprint` method. Let's create a `Case` (to be run for a two-day "spin-up" period) in this way, and then take a look at it.

In [4]:
import cstar

example_case_1 = cstar.Case.from_blueprint(blueprint  = "../examples/alpha_example/cstar_blueprint_alpha_example.yaml",
                                           caseroot   = "../examples/alpha_example/example_case", 
                                           start_date = "2012-01-01 12:00:00", 
                                           end_date   = "2012-01-03 12:00:00")

## Deconstructing the `Case` 

In [5]:
print(example_case_1)

C-Star Case
-----------
Name: roms_tools_example
caseroot: /global/cfs/cdirs/m4746/Users/dafydd/my_c_star/examples/alpha_example/example_case
start_date: 2012-01-01 12:00:00
end_date: 2012-01-03 12:00:00
Is setup: False
Valid date range:
valid_start_date: 2012-01-01 12:00:00
valid_end_date: 2012-12-31 23:00:00
This case was instantiated from the blueprint file:
   ../examples/alpha_example/cstar_blueprint_alpha_example.yaml

It is built from the following Components (query using Case.components): 
   <MARBLComponent instance>
   <ROMSComponent instance>


We can see in the printout:
- the values of the three parameters we provided (`caseroot`, `start_date`, `end_date`)
- the valid date range in which we can run this `Case`, as defined in the blueprint file (`valid_start_date`, `valid_end_date`)
- the blueprint file from which the `Case` was created
- that the `Case` is _not_ setup (yet)
- that the `Case` consists of two "`Component`"s (ROMS and MARBL)

### Looking at the `Component`s of our `Case`  
Above we saw that our `Case` consists of two `Component`s. 

The `Component` object represents is a distinct model combined with any additional code and data needed to run it in a particular configuration. Let's take a look at our two `Component` instances:
#### MARBL

In [6]:
marbl_component = example_case_1.components[0]
print(marbl_component)

MARBLComponent
--------------
base_model: MARBLBaseModel instance (query using Component.base_model)


The first entry in the `components` list is [MARBL](https://c-star.readthedocs.io/en/latest/generated/cstar.marbl.MARBLComponent.html).

Our set-up for MARBL is very simple and requires no additional code or input data, just a `BaseModel`. 

[The `BaseModel` object](https://c-star.readthedocs.io/en/latest/generated/cstar.base.BaseModel.html) represents the off-the-shelf source code for our `Component`'s model, absent any modifications:

In [7]:
print(marbl_component.base_model)

MARBLBaseModel
--------------
source_repo : https://github.com/marbl-ecosys/MARBL.git (default)
checkout_target : marbl0.45.0 (corresponding to hash 6e6b2f7c32ac5427e6cf46de4222973b8bcaa3d9)
local_config_status: 3 (Environment variable MARBL_ROOT is not present and it is assumed the base model is not installed locally)


Here we see that the base model for MARBL:
- comes from the "default" source repository (that is, the one maintained by the MARBL developers)
- Is to be checked out at version 0.45
- Is _not_ configured for use on this machine (yet)

#### ROMS
Our ROMS Component consists of much more than just a base model with no modifications. Let's take a look:

In [8]:
roms_component = example_case_1.components[1]
print(roms_component)

ROMSComponent
-------------
base_model: ROMSBaseModel instance (query using Component.base_model)
additional_source_code: AdditionalCode instance with 9 files (query using Component.additional_source_code)
namelists: AdditionalCode instance with 4 files (query using Component.namelists)
model_grid = <ROMSModelGrid instance>
initial_conditions = <ROMSInitialConditions instance>
tidal_forcing = <ROMSTidalForcing instance>
surface_forcing = <list of 2 ROMSSurfaceForcing instances>
boundary_forcing = <list of 2 ROMSBoundaryForcing instances>

Discretization:
ROMSDiscretization
------------------
time_step: 60s
n_procs_x: 3 (Number of x-direction processors)
n_procs_y: 3 (Number of y-direction processors)


Here we see that we have a base model, as before, but also:
- additional source code to be compiled alongside the base model source code
- namelist files to define certain settings at runtime
- a range of input datasets defining everything from the model grid to the surface forcing. 

Let's take a look at a few examples, but feel free to explore the other parts of the ROMS `Component` yourself:

##### Additional source code

This is managed as an [`AdditionalCode` object](https://c-star.readthedocs.io/en/latest/generated/cstar.base.AdditionalCode.html) in C-Star. We see that in this case our code is kept in a subdirectory of a GitHub repository with 9 files in it.

In [21]:
print(roms_component.additional_source_code)

AdditionalCode
--------------
Location: https://github.com/CWorthy-ocean/cstar_blueprint_roms_marbl_example.git
subdirectory: additional_code/ROMS/source_mods
Working path: None
Exists locally: False (get with AdditionalCode.get())
Files:
    bgc.opt
    bulk_frc.opt
    cppdefs.opt
    diagnostics.opt
    ocean_vars.opt
    param.opt
    tracers.opt
    Makefile
    Make.depend


We can see the full list of files using:


In [20]:
print(roms_component.additional_source_code.files)

['bgc.opt', 'bulk_frc.opt', 'cppdefs.opt', 'diagnostics.opt', 'ocean_vars.opt', 'param.opt', 'tracers.opt', 'Makefile', 'Make.depend']


##### Input datasets
Let's take a look at one of the input datasets:

In [22]:
print(roms_component.initial_conditions)

---------------------
ROMSInitialConditions
---------------------
Source location: https://github.com/CWorthy-ocean/cstar_blueprint_roms_marbl_example/raw/cstar_alpha/roms_tools_yaml_files/roms_ini.yaml
file_hash: 1786e2d4cd321a4dad04a5ea35fefb92f508776c39643da9fb78e19dcb537988
Working path: None ( does not yet exist. Call InputDataset.get() )


<div class="alert alert-info">

Note

1. The `location` attribute can either be a **local path** or a **URL**. If it is set to a URL, the `file_hash` (a 256 bit checksum) must also be provided to verify the download.
    
2. The file described by location can be either **netCDF** or **yaml** format. When C-Star sees a yaml file instead of a netCDF file for ROMS input data, it assumes the file contains a set of instructions to be passed to the [`roms-tools` package](https://roms-tools.readthedocs.io/en/latest/), which will then generate the netCDF file for us when `InputDataset.get()` is called. This makes it easier to share and save ROMS configurations without the overhead associated with potentially large netCDF files. More information on using `roms-tools` with C-Star can be found in [this notebook](https://c-star.readthedocs.io/en/latest/4_preparing_roms_input_datasets.html)

</div>

##### Discretization
Lastly, the `discretization` attribute consists of essential information for compiling and running the model - the time step and the number of processors to assign in each direction:

In [23]:
print(roms_component.discretization)

ROMSDiscretization
------------------
time_step: 60s
n_procs_x: 3 (Number of x-direction processors)
n_procs_y: 3 (Number of y-direction processors)


## Visualizing the `Case`:
We can visualize everything we've just seen using the `Case.tree()` method, which prints a representation of how this `Case` will look in the `caseroot` once set up:

In [24]:
example_case_1.tree()

/global/cfs/cdirs/m4746/Users/dafydd/my_c_star/examples/alpha_example/example_case
├── input_datasets
│   └── ROMS
│       ├── roms_grd.yaml
│       ├── roms_ini.yaml
│       ├── roms_tides.yaml
│       ├── roms_bry.yaml
│       ├── roms_bry_bgc.yaml
│       ├── roms_frc.yaml
│       └── roms_frc_bgc.yaml
├── namelists
│   └── ROMS
│       ├── roms.in_TEMPLATE
│       ├── marbl_in
│       ├── marbl_tracer_output_list
│       └── marbl_diagnostic_output_list
└── additional_source_code
    └── ROMS
        ├── bgc.opt
        ├── bulk_frc.opt
        ├── cppdefs.opt
        ├── diagnostics.opt
        ├── ocean_vars.opt
        ├── param.opt
        ├── tracers.opt
        ├── Makefile
        └── Make.depend



<div class="alert alert-info">

Note

Nothing we have seen above represents anything local on our machine (yet). each object simply describes where various files may be found, and the `Case.tree()` representation shows us where these files will be assembled once we set everything up. To turn that into something concrete we can work with, we call `Case.setup()`.

</div>

## Setting up the `Case` :

Next we call `Case.setup()`. This will:
- Fetch and compile our base models (ROMS and MARBL)
- Fetch any remote data associated with this case
- Construct any ROMS netCDF files from yaml files using `roms-tools` 

We will be prompted before installing the base models, so some input is required here:

In [26]:
example_case_1.setup()


Setting up MARBLComponent
--------------------------
Configuring MARBLComponent
--------------------------
#######################################################
C-STAR: MARBL_ROOT not found in current environment. 
if this is your first time running C-Star with an instance of MARBLBaseModel, you will need to set it up.
It is recommended that you install this base model in 
/global/cfs/cdirs/m4746/Users/dafydd/my_c_star/cstar/externals/MARBL
This will also modify your `cstar_local_config.py` file.
#######################################################


Would you like to do this now? ('y', 'n', or 'custom' to install at a custom path)
 y


Cloned repository https://github.com/marbl-ecosys/MARBL.git to /global/cfs/cdirs/m4746/Users/dafydd/my_c_star/cstar/externals/MARBL
Checked out marbl0.45.0 in git repository /global/cfs/cdirs/m4746/Users/dafydd/my_c_star/cstar/externals/MARBL
Updating environment in C-Star configuration file /global/cfs/cdirs/m4746/Users/dafydd/my_c_star/cstar/cstar_local_config.py
Compiling MARBL...
MARBL successfully installed at /global/cfs/cdirs/m4746/Users/dafydd/my_c_star/cstar/externals/MARBL

Setting up ROMSComponent
-------------------------
Configuring ROMSComponent
-------------------------
#######################################################
C-STAR: ROMS_ROOT not found in current environment. 
if this is your first time running C-Star with an instance of ROMSBaseModel, you will need to set it up.
It is recommended that you install this base model in 
/global/cfs/cdirs/m4746/Users/dafydd/my_c_star/cstar/externals/ucla-roms
This will also modify your `cstar_local_config.py` file.
#########

Would you like to do this now? ('y', 'n', or 'custom' to install at a custom path)
 y


Cloned repository https://github.com/CESR-lab/ucla-roms.git to /global/cfs/cdirs/m4746/Users/dafydd/my_c_star/cstar/externals/ucla-roms
Checked out 594ac425e9dbe663ce48ced0915c0007c6cca843 in git repository /global/cfs/cdirs/m4746/Users/dafydd/my_c_star/cstar/externals/ucla-roms
Compiling UCLA ROMS' NHMG library...
Compiling Tools-Roms package for UCLA ROMS...
UCLA-ROMS is installed at /global/cfs/cdirs/m4746/Users/dafydd/my_c_star/cstar/externals/ucla-roms

Fetching additional source code...
----------------------------------
Cloned repository https://github.com/CWorthy-ocean/cstar_blueprint_roms_marbl_example.git to /tmp/tmpyljufp32
Checked out cstar_alpha in git repository /tmp/tmpyljufp32
copying bgc.opt to /global/cfs/cdirs/m4746/Users/dafydd/my_c_star/examples/alpha_example/example_case/additional_source_code/ROMS
copying bulk_frc.opt to /global/cfs/cdirs/m4746/Users/dafydd/my_c_star/examples/alpha_example/example_case/additional_source_code/ROMS
copying cppdefs.opt to /global/cf

## Compiling the `Case` and performing pre-processing
We have now assembled all the data we need to run this `Case` in one place. Lastly, we need to compile the additional code we've obtained and run some pre-processing steps on the input data:

In [27]:
#The Case.build() method compiles the code:
example_case_1.build()


Compiling MARBLComponent
-------------------------
No build steps to be completed for MARBLComponent

Compiling ROMSComponent
------------------------
Compiling UCLA-ROMS configuration...
UCLA-ROMS compiled at /global/cfs/cdirs/m4746/Users/dafydd/my_c_star/examples/alpha_example/example_case/additional_source_code/ROMS


In [28]:
# The Case.pre_run() method performs pre-processing:
example_case_1.pre_run()


Completing pre-processing steps for MARBLComponent
---------------------------------------------------
No pre-processing steps to be completed for MARBLComponent

Completing pre-processing steps for ROMSComponent
--------------------------------------------------
Partitioning /global/cfs/cdirs/m4746/Users/dafydd/my_c_star/examples/alpha_example/example_case/input_datasets/ROMS/roms_grd.nc into (3,3)
Partitioning /global/cfs/cdirs/m4746/Users/dafydd/my_c_star/examples/alpha_example/example_case/input_datasets/ROMS/roms_ini.nc into (3,3)
Partitioning /global/cfs/cdirs/m4746/Users/dafydd/my_c_star/examples/alpha_example/example_case/input_datasets/ROMS/roms_tides.nc into (3,3)
Partitioning /global/cfs/cdirs/m4746/Users/dafydd/my_c_star/examples/alpha_example/example_case/input_datasets/ROMS/roms_bry_201201.nc into (3,3)
Partitioning /global/cfs/cdirs/m4746/Users/dafydd/my_c_star/examples/alpha_example/example_case/input_datasets/ROMS/roms_bry_bgc_clim.nc into (3,3)
Partitioning /global/c

## Running the `Case` :

In [29]:
example_case_1.run(account_key="m4746", walltime="00:10:00", queue="shared")


Running ROMS: 
------------
Submitted batch job 31761927


### C-Star currently doesn't support monitoring of jobs handled by a scheduler, so we can use a bash cell to do this manually:
Things can sit in the queue on Perlmutter for a while, so this might be a good time for a break

In [30]:
%%bash
squeue -u $USER

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
          31761927 shared_mi my_case_   dafydd PD       0:00      1 (Priority)
          31756993 urgent_mi  jupyter   dafydd  R    2:32:00      1 nid004182


## Post-processing
Once the run is complete, we can carry out any post-processing steps.
When ROMS runs on multiple CPUs in parallel, it produces one output file per CPU. To work with the output, we thus need to join these files together. This is handled with the `post_run()` method:

In [18]:
example_case_1.post_run()


Completing post-processing steps for ROMSComponent
---------------------------------------------------
Joining netCDF files ROMS_MARBL_rst.20120102120000.*.nc...
Joining netCDF files ROMS_MARBL_his.20120101120000.*.nc...
Joining netCDF files ROMS_MARBL_bgc_dia.20120101120000.*.nc...
Joining netCDF files ROMS_MARBL_rst.20120103120000.*.nc...
Joining netCDF files ROMS_MARBL_his.20120103120000.*.nc...
Joining netCDF files ROMS_MARBL_bgc.20120101120000.*.nc...
Joining netCDF files ROMS_MARBL_his.20120102120000.*.nc...


## Summary

In this notebook we:
- created a C-Star `Case` from a "blueprint" file
- Ran the case for 2 days from 2012-01-01 to 2012-01-03 with a 60 second time-step
- Restarted the case from 2012-01-03 and ran for a further 3 days with a 6 minute time-step
- Produced a basic plot to verify the output